# 🤖 Vision Transformer (ViT) 소개

> **학습 목표**: ViT 아키텍처를 이해하고 pre-trained 모델로 이미지 분류 실습

---

## 📋 학습 내용

1. ✅ ViT 아키텍처 개념 (Patch Embedding, Position Encoding)
2. ✅ Self-Attention Mechanism 시각화
3. ✅ Pre-trained ViT 모델 추론
4. ✅ ResNet50 vs ViT-Base 성능 비교
5. ✅ Multi-scale Inference (224, 384, 512)
6. ✅ Attention Map 시각화 (12 layers)

**소요 시간**: 약 45분  
**난이도**: ⭐⭐⭐ (중상급)  
**사전 지식**: CNN 기초, Transformer 개념

---

## 🔧 Step 1: 라이브러리 Import

In [ ]:
# Transformer 라이브러리
try:
    from transformers import ViTForImageClassification, ViTImageProcessor, ViTConfig
    import transformers
    print(f"✅ Transformers 버전: {transformers.__version__}")
except ImportError:
    print("❌ Transformers 설치 필요: pip install transformers")

# 이미지 처리
from PIL import Image
import numpy as np

# 딥러닝
import torch
import torch.nn.functional as F
print(f"✅ PyTorch 버전: {torch.__version__}")
print(f"✅ GPU 사용 가능: {torch.cuda.is_available()}")

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 유틸리티
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (14, 6)

print("\n✅ 라이브러리 로드 완료!")

## 📂 Step 2: 데이터 로드

**Dataset_Machinevision.zip** 압축 해제 후 사용

In [ ]:
# 데이터 디렉토리 설정
# 1순위: KAMP 실제 데이터 (시지트로닉스 이미지)
# 2순위: 로컬 data/ 폴더
# 3순위: 합성 샘플 이미지 자동 생성

kamp_dir = Path('../../dataset/part2-2/시지트로닉스_이미지')
local_dir = Path('../data/MachineVision')

if kamp_dir.exists() and any(kamp_dir.iterdir()):
    data_dir = kamp_dir
    data_source = "KAMP 실제 데이터"
elif local_dir.exists() and any(local_dir.rglob('*.png')) or local_dir.exists() and any(local_dir.rglob('*.jpg')):
    data_dir = local_dir
    data_source = "로컬 데이터"
else:
    data_dir = local_dir
    data_source = "합성 샘플 (자동 생성)"

# 이미지 파일 검색
image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp']
image_files = []
for ext in image_extensions:
    image_files.extend(list(data_dir.rglob(ext)))

print(f"📂 데이터 경로: {data_dir}")
print(f"📦 데이터 소스: {data_source}")
print(f"🖼️ 이미지 파일 수: {len(image_files):,}개")

# 샘플 이미지가 없으면 생성
if len(image_files) == 0:
    print("\n⚠️ 이미지 파일이 없습니다. 샘플 이미지 생성...")
    data_dir.mkdir(exist_ok=True, parents=True)
    
    # 랜덤 샘플 이미지 생성
    img_array = np.random.randint(0, 256, (224, 224, 3), dtype=np.uint8)
    img = Image.fromarray(img_array)
    sample_path = data_dir / 'sample.png'
    img.save(sample_path)
    image_files = [sample_path]
    print(f"✅ 샘플 이미지 생성: {sample_path}")
else:
    print(f"\n📁 디렉토리 구조:")
    subdirs = sorted(set([f.parent.name for f in image_files]))
    for subdir in subdirs[:10]:
        count = len([f for f in image_files if f.parent.name == subdir])
        print(f"   - {subdir}: {count:,}개")

In [ ]:
# 샘플 이미지 로드 및 시각화
sample_images = image_files[:6]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, img_path in enumerate(sample_images):
    img = Image.open(img_path).convert('RGB')
    axes[idx].imshow(img)
    axes[idx].set_title(f'{img_path.parent.name}\n{img.size[0]}×{img.size[1]}', fontsize=10)
    axes[idx].axis('off')

plt.suptitle('샘플 이미지 미리보기', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"\n✅ 샘플 이미지 {len(sample_images)}개 로드 완료")

## 🤖 Step 3: ViT 모델 로드

**google/vit-base-patch16-224**: ImageNet-21k pre-trained, ImageNet-1k fine-tuned

In [ ]:
# ViT 모델 및 프로세서 로드
model_name = 'google/vit-base-patch16-224'

print(f"🔄 ViT 모델 로딩 중: {model_name}...")

try:
    model = ViTForImageClassification.from_pretrained(model_name)
    processor = ViTImageProcessor.from_pretrained(model_name)
    config = model.config
    
    print(f"\n✅ ViT 모델 로드 완료!")
    print(f"\n📊 모델 구조:")
    print(f"   - 파라미터 수: {model.num_parameters():,}개")
    print(f"   - Hidden size: {config.hidden_size}")
    print(f"   - Attention heads: {config.num_attention_heads}")
    print(f"   - Transformer layers: {config.num_hidden_layers}")
    print(f"   - Patch size: {config.patch_size}×{config.patch_size}")
    print(f"   - Image size: {config.image_size}×{config.image_size}")
    print(f"   - 클래스 수: {config.num_labels}")
    
    # 패치 수 계산
    num_patches = (config.image_size // config.patch_size) ** 2
    print(f"   - Patch 개수: {num_patches}개 ({config.image_size // config.patch_size}×{config.image_size // config.patch_size})")
    
except Exception as e:
    print(f"❌ 모델 로드 실패: {e}")
    print("   transformers 설치: pip install transformers")

# GPU 사용 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()
print(f"\n🖥️ 디바이스: {device}")

## 🔍 Step 4: Patch Embedding 시각화

이미지를 16×16 패치로 분할하고 임베딩하는 과정 시각화

In [ ]:
# 샘플 이미지 선택
sample_img = Image.open(image_files[0]).convert('RGB')
sample_img = sample_img.resize((224, 224))

# 패치 분할 시각화
patch_size = config.patch_size
img_array = np.array(sample_img)

fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# 원본 이미지
axes[0, 0].imshow(img_array)
axes[0, 0].set_title('원본 이미지 (224×224)', fontsize=12, fontweight='bold')
axes[0, 0].axis('off')

# 패치 그리드 오버레이
axes[0, 1].imshow(img_array)
for i in range(0, 224, patch_size):
    axes[0, 1].axhline(i, color='red', linewidth=1, alpha=0.5)
    axes[0, 1].axvline(i, color='red', linewidth=1, alpha=0.5)
axes[0, 1].set_title(f'Patch Grid ({patch_size}×{patch_size})', fontsize=12, fontweight='bold')
axes[0, 1].axis('off')

# 개별 패치 샘플 (4개 — 2×3 그리드의 남은 4칸에 배치)
patch_indices = [(0, 0), (0, 7), (7, 0), (7, 7)]

for idx, (row, col) in enumerate(patch_indices):
    ax_row = (idx + 2) // 3
    ax_col = (idx + 2) % 3
    ax = axes[ax_row, ax_col]
    
    y_start = row * patch_size
    x_start = col * patch_size
    patch = img_array[y_start:y_start+patch_size, x_start:x_start+patch_size]
    
    ax.imshow(patch)
    ax.set_title(f'Patch ({row},{col})', fontsize=10)
    ax.axis('off')

plt.suptitle('ViT Patch Embedding 과정', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"\n💡 Patch Embedding 설명:")
print(f"   1. 이미지를 {patch_size}×{patch_size} 패치로 분할 → {num_patches}개 패치")
print(f"   2. 각 패치를 1D 벡터로 flatten → {patch_size * patch_size * 3}차원")
print(f"   3. Linear projection → {config.hidden_size}차원 임베딩")
print(f"   4. Position encoding 추가 → Transformer 입력")

## 📊 Step 5: 추론 및 예측

In [ ]:
# 이미지 전처리
inputs = processor(images=sample_img, return_tensors='pt')
inputs = {k: v.to(device) for k, v in inputs.items()}

# 추론
with torch.no_grad():
    outputs = model(**inputs, output_attentions=True)
    logits = outputs.logits
    attentions = outputs.attentions  # 12 layers attention weights

# 예측 결과
probs = F.softmax(logits, dim=-1)
top5_probs, top5_indices = torch.topk(probs, k=5)

print("🎯 예측 결과 (Top 5):")
print(f"{'순위':<5} {'클래스 ID':<10} {'신뢰도':<10}")
print("-" * 30)

for rank, (prob, idx) in enumerate(zip(top5_probs[0], top5_indices[0]), 1):
    print(f"{rank:<5} {idx.item():<10} {prob.item():.2%}")

predicted_class = top5_indices[0][0].item()
confidence = top5_probs[0][0].item()

print(f"\n✅ 최종 예측: 클래스 {predicted_class} (신뢰도: {confidence:.2%})")
print(f"\n📊 Attention 정보:")
print(f"   - Attention layers: {len(attentions)}개")
print(f"   - Attention shape: {attentions[0].shape}")  # (batch, heads, seq, seq)
print(f"   - Attention heads: {attentions[0].shape[1]}개")

## 🔍 Step 6: Attention Map 시각화

12개 Transformer layer의 attention pattern 시각화

In [ ]:
# Attention map 시각화 (첫 번째 헤드, CLS 토큰에 대한 attention)
fig, axes = plt.subplots(3, 4, figsize=(16, 12))
axes = axes.flatten()

for layer_idx in range(12):
    # CLS 토큰 (첫 번째 토큰)의 attention
    attention = attentions[layer_idx][0, 0, 0, 1:].detach().cpu().numpy()  # [0, head=0, cls_token, patches]
    
    # Reshape to 2D grid
    grid_size = int(np.sqrt(len(attention)))
    attention_map = attention.reshape(grid_size, grid_size)
    
    # 시각화
    im = axes[layer_idx].imshow(attention_map, cmap='hot', interpolation='nearest')
    axes[layer_idx].set_title(f'Layer {layer_idx + 1}', fontsize=10, fontweight='bold')
    axes[layer_idx].axis('off')

plt.suptitle('ViT Attention Maps (CLS Token → Patches)', fontsize=14, fontweight='bold', y=1.02)
plt.colorbar(im, ax=axes, fraction=0.046, pad=0.04)
plt.tight_layout()
plt.show()

print("\n💡 Attention Map 해석:")
print("   - 밝은 영역: 예측에 중요한 패치 (높은 attention weight)")
print("   - 어두운 영역: 덜 중요한 패치 (낮은 attention weight)")
print("   - Layer가 깊어질수록 attention이 특정 영역에 집중하는 경향")

## ⚖️ Step 7: Multi-scale Inference

다양한 이미지 크기(224, 384, 512)로 추론 속도 및 성능 비교

In [ ]:
import time

# 다양한 이미지 크기로 추론
scales = [224, 384, 512]
results = []

print("🔄 Multi-scale Inference 실행 중...\n")

for size in scales:
    # 이미지 리사이즈
    img_resized = sample_img.resize((size, size))
    
    # 전처리
    inputs = processor(images=img_resized, return_tensors='pt')
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # 추론 시간 측정
    start_time = time.time()
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
    
    inference_time = (time.time() - start_time) * 1000  # ms
    
    # 예측 결과
    probs = F.softmax(logits, dim=-1)
    top_prob, top_idx = torch.max(probs, dim=-1)
    
    # 패치 수 계산
    num_patches_scale = (size // config.patch_size) ** 2
    
    results.append({
        'size': size,
        'patches': num_patches_scale,
        'class': top_idx.item(),
        'confidence': top_prob.item(),
        'time_ms': inference_time
    })
    
    print(f"📐 크기 {size}×{size}:")
    print(f"   - Patch 수: {num_patches_scale}개")
    print(f"   - 예측 클래스: {top_idx.item()}")
    print(f"   - 신뢰도: {top_prob.item():.2%}")
    print(f"   - 추론 시간: {inference_time:.2f} ms\n")

# 결과 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 추론 시간
axes[0].bar([str(r['size']) for r in results], [r['time_ms'] for r in results], color='skyblue')
axes[0].set_title('Inference Time (ms)', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Image Size')
axes[0].set_ylabel('Time (ms)')
axes[0].grid(alpha=0.3)

# 신뢰도
axes[1].bar([str(r['size']) for r in results], [r['confidence'] for r in results], color='coral')
axes[1].set_title('Prediction Confidence', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Image Size')
axes[1].set_ylabel('Confidence')
axes[1].set_ylim(0, 1)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Multi-scale 분석:")
print("   - 이미지 크기가 클수록 패치 수 증가 → 추론 시간 증가")
print("   - 더 많은 패치로 더 세밀한 특징 추출 가능")
print("   - 실시간 추론이 필요하면 224×224, 정확도가 중요하면 384 이상 권장")

## 💾 Step 8: 결과 저장

In [ ]:
import pandas as pd

# 출력 디렉토리 생성
output_dir = Path('../outputs')
output_dir.mkdir(exist_ok=True)

# Multi-scale 결과 저장
results_df = pd.DataFrame(results)
results_file = output_dir / '01_vit_multiscale_results.csv'
results_df.to_csv(results_file, index=False, encoding='utf-8-sig')
print(f"✅ Multi-scale 결과 저장: {results_file}")

# 모델 구조 정보 저장
model_info = pd.DataFrame({
    'parameter': ['total_params', 'hidden_size', 'attention_heads', 'layers', 'patch_size', 'image_size', 'num_patches'],
    'value': [
        model.num_parameters(),
        config.hidden_size,
        config.num_attention_heads,
        config.num_hidden_layers,
        config.patch_size,
        config.image_size,
        num_patches
    ]
})
model_info_file = output_dir / '01_vit_model_info.csv'
model_info.to_csv(model_info_file, index=False, encoding='utf-8-sig')
print(f"✅ 모델 정보 저장: {model_info_file}")

# Attention weights 저장 (첫 번째 layer)
attention_weights = attentions[0][0, 0].detach().cpu().numpy()  # (seq, seq)
attention_file = output_dir / '01_vit_attention_layer1.npy'
np.save(attention_file, attention_weights)
print(f"✅ Attention weights 저장: {attention_file}")

print("\n🎉 ViT 소개 완료!")

---

## 🎯 학습 정리

### ✅ 완료한 내용
1. ViT 아키텍처 이해 및 모델 로드 (86M parameters)
2. Patch Embedding 과정 시각화 (16×16 patches)
3. Pre-trained 모델로 이미지 분류 추론
4. 12-layer Attention Map 시각화
5. Multi-scale Inference (224, 384, 512) 성능 비교

### 💡 핵심 인사이트

- **ViT의 핵심 아이디어**:
  - 이미지를 패치로 분할하여 시퀀스로 처리 (NLP Transformer 활용)
  - CNN 대신 Self-Attention으로 전역적 관계 학습
  - Position Encoding으로 패치의 공간적 위치 정보 보존

- **ViT vs CNN**:
  - CNN: Inductive bias (locality, translation invariance) 내장
  - ViT: 더 많은 데이터 필요, 하지만 대규모 데이터셋에서 더 높은 성능
  - ViT는 Long-range dependency 학습에 유리

- **Attention 분석**:
  - 초기 레이어: 전역적으로 고르게 분산
  - 깊은 레이어: 객체 영역에 집중
  - CLS 토큰이 모든 패치 정보를 집약하여 최종 분류

- **실무 활용**:
  - Pre-trained ViT로 Transfer Learning (다음 실습에서)
  - 제조 불량 검사, 의료 이미지 분석 등
  - Attention map으로 모델 해석 가능 (XAI)

### 📚 다음 단계
- **02_vit_transfer_learning.ipynb**: ViT Fine-tuning으로 제조 불량 분류
- **03_yolov8_finetuning.ipynb**: YOLOv8 Object Detection

### 🔗 참고 자료
- [ViT 논문](https://arxiv.org/abs/2010.11929) - An Image is Worth 16x16 Words
- [Hugging Face ViT](https://huggingface.co/docs/transformers/model_doc/vit)
- [Transformers 공식 문서](https://huggingface.co/docs/transformers/)

---

*제조AI 교육 v12 Enhanced | Part 2-2 | 2025.02*